##3. Escoged a uno de los dos presidentes, y escribid tweets como ellos, usando un Modelo Generativo. Donald Trump.


In [0]:
#Importar librerías

import spacy
import numpy as np

import pickle
import json
import os
import csv
import pprint as pp

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from random import shuffle, choice, sample

from sklearn.model_selection import StratifiedShuffleSplit

from copy import copy

import warnings
warnings.filterwarnings('ignore')

nlp = spacy.load('en_core_web_sm')

import pylab as pl
from IPython import display

sns.set(color_codes=True)

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
# Importar librerías

from keras.models import Model, Sequential
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dot, Concatenate, Flatten, Permute, Multiply, dot, concatenate
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

Using TensorFlow backend.


## Preprocess

In [3]:
from google.colab import files
import io
import json

import numpy as np 
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#Crea un diccionario con los tweets de Trump. 
dataS = json.loads((open('/content/drive/My Drive/BootCamp - BigDataIV - NLP/data_practica_realDonaldTrump.json').read()))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Convertir diccionario en dataframe de registros. 
df = pd.DataFrame([[key, dataS[key]] for key in dataS.keys()], columns=['Id', 'TrumpTweet'])

In [5]:
df

,Id,TrumpTweet
0,0,My thoughts and prayers are with the families ...
1,1,My thoughts and prayers are with the families ...
2,2,I am heading for Canada and the G-7 for talks ...
3,3,Congratulations to the Washington Capitals on ...
4,4,Looking forward to straightening out unfair Tr...
...,...,...
2990,2990,...vast sums of money to NATO &amp; the United...
2991,2991,Despite what you have heard from the FAKE NEWS...
2992,2992,Great meeting with the @RepublicanStudy Commit...
2993,2993,"""The President Changed. So Has Small Businesse..."


In [0]:
# Crea una lista de 2994 registros con el número de índice y el texto del tweet
dataset = list()
for index, row in df.iterrows():  # para cada registro
    if index > 0:
        sentence = row[1]         # toms el texto de cada registro
        dataset.append(sentence)  # añade el texto del tweet al dataset

In [0]:
tokenized = [list(x) for x in dataset] # Convierte cada clase en tokens por caracteres

In [0]:
init_chars = [x[:5] for x in tokenized] # me quedo con los 4 caracteres iniciales

for i in range(len(init_chars)): # genera lista de tokens de los cinco primeros de cada frase
    tmp = init_chars[i]
    tmp.insert(0, '<SOS>') # añadiendo el token especial SOS
    init_chars[i] = tmp[:5]

In [9]:
maxlen = max([len(x) for x in tokenized])  # max longitud de cada clase tokenizada por caracteres       
avglen = sum([len(x) for x in tokenized]) / len(tokenized) # media de caracteres por frase en los tweets
maxlen, avglen

(320, 160.39111556446227)

In [10]:
total_tokens = [t for s in dataset for t in s] # total de caracteres
len(total_tokens)

480211

In [11]:
from collections import Counter
vocab_counter = Counter(total_tokens) # Cuenta el número de veces que aparece cada carácter
vocab = [w for w, v in vocab_counter.items() if v>2] # filtrado de caracteres que minimo hayan aparecido 2 veces
vocab = ['<PAD>', '<UNK>', '<SOS>', '<EOS>'] + vocab # añade al vocabulario los tokens especiales
nb_vocab = len(vocab)
nb_vocab

111

In [0]:
c2id = {k: i for i, k in enumerate(vocab)} # asigna a cada token un número
id2c = {i: k for k, i in c2id.items()}     # crea un diccionario

In [13]:
maxlen = 5
step = 1
data_train = []                # dataset al que iremos añadiendo

for x in tokenized:            # x es una lista con mi frase "tokenizada"
    x.insert(0, '<SOS>')       # se le inserta al principio de cada frase SOS
    x.append('<EOS>')          # se le inserta al final de cada frase EOS
    for i in range(0, len(x)-maxlen, step): # longitud de cada frase tokenizada-5
        # se añade a la lista un conjunto de 1-5 primeros caracteres de cada frase, luego 2-6, 3-7...
        data_train.append((x[i:i+maxlen], x[i+maxlen])) 
len(data_train) # esta lista tiene 471.229

471229

In [0]:
SAMPLE_EVERY = 4

Callbacks : una devolución de llamada o retrollamada es una función "A" que se usa como argumento de otra función "B". Cuando se llama a "B", ésta ejecuta "A". Para conseguirlo, usualmente lo que se pasa a "B" es el puntero a "A".

In [0]:
# función que ayuda a que las predicciones de los caracteres ganen diversidad 
def sample_pred(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
class Sampletest(Callback):
# cuando termine la epoch de entreno, me da una muestra de qué texto está aprendiendo, y el progreso a nivel de 
# frases generadas
    def on_epoch_end(self, epoch, logs):  
        if epoch % SAMPLE_EVERY == 0  and epoch>0:
            data_test = []
            nb_samples = 1
            
            params = {
                'maxlen': maxlen,
                'vocab': len(vocab),
                'use_embeddings': True
                }
            for _ in range(nb_samples):
                data_test = choice(init_chars)
                for diversity in [0.2, 0.6, 1.2]:
                    print('----- diversity:', diversity)
                    sentence = copy(data_test)
                    generated = copy(data_test)
                    for i in range(len(data_test), 400):
                        x_pred = np.zeros((1, params['maxlen']))
                        for t, char in enumerate(sentence):
                            x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                        preds = self.model.predict(x_pred, verbose=0)[0]
                        next_index = sample_pred(preds, diversity)
                        next_char = id2c[next_index]
                        if next_char == '<EOS>':
                            break
                        generated += [next_char]
                        sentence = sentence[1:] 
                        sentence += [next_char]
                    print(''.join(generated))  # muestra por pantalla la frase generada


In [0]:
class HistoryDisplay(Callback):
# Muestra una gráfica de los accuracy y loss    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % PLOT_EVERY == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()
        

In [0]:
class TimeHistory(Callback):
# Devuelve lo que tarda cada epoch
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

# Arquitectura y preparar el train y el predict

In [0]:
class LM:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
        
    def compile_model(self, params={}):
        # capa de input 
        lm_inputs = Input(shape=(params['maxlen'], ), name='lm_input')
        # para vincular palabras a vectores
        embedding = Embedding(params['vocab'], params['emb_feats'])(lm_inputs)
        # return sequences devuelve el resultado por cada w-emdedding por separado  
        lstm = CuDNNLSTM(params['rnn_hidden'], return_sequences=True, name='rnn1')

        lmlstm = Bidirectional(lstm)(embedding)
        
        stacklstm = CuDNNLSTM(params['rnn_hidden'], return_sequences=False, name='stack')
        stackedlstm = stacklstm(lmlstm)
        #capa de salida
        out = Dense(params['vocab'], activation='softmax')(stackedlstm)

        model = Model(lm_inputs, out)
        model.compile(
            optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        model.summary()
        return model
        
        
    def train(self, model, data, params={}):
        callbacks = self._get_callbacks()   #llamada a los callbacks de arriba

        if 'shuffle' in params and params['shuffle']: #si la clave está en dicc de parámetros y es true
            shuffle(data)

        sentences, next_chars = zip(*data)
        x = np.zeros((len(data), params['maxlen']))
        y = np.zeros((len(data), params['vocab']))
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t] = c2id[char] if char in c2id else c2id['<UNK>']
            y[i, c2id[next_chars[i]] if next_chars[i] in c2id else c2id['<UNK>']] = 1

        model.fit(x, y, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks, verbose=1)
        
    def predict(self, model, data, params={}):
        for diversity in [0.2, 0.6, 1.2]:
            print('------ diversity: ', diversity)
            sentence = copy(data)
            generated = copy(data)
            for i in range(len(data), 240): # si fuese un tweet
                x_pred = np.zeros((1, params['maxlen']))
                for t, char in enumerate(sentence):
                    x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                
                preds = self.model.predict(x_pred, verbose=0)[0]
                next_index = sample_pred(preds, diversity)
                next_char = id2c[next_index]

                if next_char == '<EOS>':
                    break
                
                generated += [next_char]
                sentence = sentence[1:]
                sentence+= [next_char]
            
            print(''.join(generated))
        
    def _get_callbacks(self, model_path='model_lm.h5'):
     # monitoriza train y validation loss. Si en la 4ºepoch validation loss no baja, el train se detiene
        es = EarlyStopping(monitor='loss', patience=4, mode='auto', verbose=0)
     # cada vez que baja la validation loss guardará una copia del modelo. Machaca modelos peores. No guarda pesos.
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, period=2)
     # reduce el learning rate si durante 3 epochs no ha bajado
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)
        st = Sampletest()
        hd = HistoryDisplay()
        return [st, rlr, es]

Hiperparametros y compilación.

In [0]:
compile_params = {
    'maxlen': maxlen,
    'vocab': len(vocab),
    'emb_feats': 100,
    'rnn_hidden': 256
}

In [21]:
lm = LM()

lm_model = lm.compile_model(params=compile_params)






Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lm_input (InputLayer)        (None, 5)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5, 100)            11100     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 5, 512)            733184    
_________________________________________________________________
stack (CuDNNLSTM)            (None, 256)               788480    
_________________________________________________________________
dense_1 (Dense)              (None, 111)               28527     
Total params: 1,561,291
Trainable params: 1,561,291
Non-trainable params: 0
_________________________________________________________________


Entrenar. Genera frases.

In [0]:
train_params = {
    'epochs': 500,
    'batch_size': 512,
    'shuffle': True,
    'vocab': len(vocab),
    'maxlen': maxlen,
}

In [0]:
lm.train(lm_model, data=data_train, params=train_params)
# Genera frases como si las dijera Donald Trump (la frase empieza en SOS) cada 6 epochs

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/500





471229/471229 [==============================] - 28s 59us/step - loss: 2.4908 - acc: 0.3304
Epoch 2/500
471229/471229 [==============================] - 11s 24us/step - loss: 1.8835 - acc: 0.4897
Epoch 3/500
471229/471229 [==============================] - 11s 24us/step - loss: 1.6449 - acc: 0.5530
Epoch 4/500
471229/471229 [==============================] - 11s 24us/step - loss: 1.5161 - acc: 0.5849
Epoch 5/500
471229/471229 [==============================] - 11s 24us/step - loss: 1.4338 - acc: 0.6048
----- diversity: 0.2
<SOS>It was any with the was an and the Democrats and working in the U.S.A.....
----- diversity: 0.6
<SOS>It was as a truly was an encourage in their hard to stand the coming great the was an tremendous standing and and going very collusion which we are with the Fake News and continues to help on the Democrats and doing very well.
----- diversity: 1.2
<SO